In [2]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import zipfile
from helper import spkly

In [5]:
zip = zipfile.ZipFile('../data/Teleost_Especes_Pelagiques_Abondance_2009-2018.zip')
zip.namelist()

# let's get the fish mappings - requires specific encoding to be read
FISH_MAP = pd.read_csv('../data/DataDictionary_DictionnaireDonnees.csv', encoding='cp437', skiprows=10).set_index('Species')
FISH_MAP = FISH_MAP[['Name _EN']].to_dict('dict')['Name _EN']
FISH_MAP

{'AMSPP': 'Sand Lances (Ammodytes sp.)',
 'ARRIS': 'White Barracudina',
 'ARSIL': 'Atlantic Argentine',
 'BOSAI': 'Arctic Cod',
 'CLHAR': 'Atlantic Herring',
 'CYLUM': 'Lumpfish',
 'GAACU': 'Threespine Stickleback',
 'MAVIL': 'Capelin (M. villosus)',
 'MEATL': 'Atlantic Soft Pout',
 'MEBIL': 'Silver Hake',
 'OSMOR': 'Rainbow Smelt',
 'POVIR': 'Pollock',
 'SCSCO': 'Atlantic Mackerel (S. scombrus)'}

In [10]:
import datetime 

north_fish, south_fish = pd.read_csv(zip.open(zip.namelist()[0])), pd.read_csv(zip.open(zip.namelist()[1]))
north_fish['region'] = 'NORTH'
south_fish['region'] = 'SOUTH'

# let's just append the two datasets together and rename the fish mappings to something slightly more human readable 
fish_set = pd.concat([north_fish, south_fish]).rename(columns=FISH_MAP).copy()
fish_set['Date'] = pd.to_datetime(fish_set['Date'])
display(fish_set.shape)

# these sparkline image volumes don't look very promising for some species... e.g., sand lances. Also see that this data is only collected 3
# months out of a year!
spkly.display(fish_set.groupby([fish_set['Date'].dt.year, fish_set['Date'].dt.month]).sum())

(3165, 19)

In [14]:
fish_set.dtypes

Date                               datetime64[ns]
Station                                     int64
Longitude                                 float64
Latitude                                  float64
Profondeur/Depth                          float64
Sand Lances (Ammodytes sp.)               float64
White Barracudina                         float64
Atlantic Argentine                        float64
Arctic Cod                                float64
Atlantic Herring                          float64
Lumpfish                                  float64
Threespine Stickleback                    float64
Capelin (M. villosus)                     float64
Atlantic Soft Pout                        float64
Silver Hake                               float64
Rainbow Smelt                             float64
Pollock                                   float64
Atlantic Mackerel (S. scombrus)           float64
region                                     object
dtype: object

In [ ]:
import folium

import folium
m = folium.Map(location=[50, -60], zoom_start=4.5, tiles='Stamen Terrain')

for flt in fish_set.index[:10]:
    folium.Marker(location=[fish_set["Latitude"].iloc[flt], fish_set["Longitude"].iloc[flt]]).add_to(m)
m